In [ ]:
!wget https://s3.amazonaws.com/ifashionist-dataset/images/train2020.zip
!wget https://s3.amazonaws.com/ifashionist-dataset/images/val_test2020.zip
!wget https://s3.amazonaws.com/ifashionist-dataset/annotations/instances_attributes_train2020.json
!wget https://s3.amazonaws.com/ifashionist-dataset/annotations/instances_attributes_val2020.json
!wget https://s3.amazonaws.com/ifashionist-dataset/annotations/info_test2020.json

In [ ]:
import pandas as pd
import json
datatrain=json.load(open('/content/instances_attributes_train2020.json'))
dataval=json.load(open('/content/instances_attributes_val2020.json'))

In [ ]:
annot_train=pd.DataFrame(datatrain['annotations'])
annot_val=pd.DataFrame(dataval['annotations'])
annot_train.head()

In [ ]:
image_attri_train=pd.DataFrame(datatrain['images'])
image_attri_val=pd.DataFrame(dataval['images'])
image_attri_train.head()

In [ ]:
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/train/images
!mkdir dataset/train/labels
!mkdir dataset/val
!mkdir dataset/val/images
!mkdir dataset/val/labels

In [ ]:
!unzip -xq train2020.zip
!unzip -xq val_test2020.zip
!mv test val

In [ ]:
import shutil
def create_txt(annot,attrip,typ):
  for i in annot['image_id'].unique():
    image_annotaions=annot[annot['image_id']==i]
    attriputes=attrip[attrip['id']==i]
    img_name=attrip[attrip['id']==i]['file_name']
    img_name=img_name[img_name.keys()[0]]
    width,height=int(attriputes['width']),int(attriputes['height'])
    lines=[]
    for cat,seg in zip(image_annotaions.category_id,image_annotaions.segmentation):
      if cat not in range(0,12):
        continue
      try:
        segs=' '.join([str(int(elem)/width) if count%2==0 else str(int(elem)/height) for count,elem in
                      enumerate(str(seg).replace('[','').replace(']','').replace(' ','').split(','))])
        lines.append('{} {}\n'.format(cat,segs))
      except:
        pass
    if len(lines)>0:
      with open('dataset/{}/labels/{}'.format(typ,img_name.replace('.jpg','.txt')),'w') as f:
        f.writelines(lines)
        f.close()
      shutil.copy('{}/{}'.format(typ,img_name),'dataset/{}/images/{}'.format(typ,img_name))
create_txt(annot_train,image_attri_train,'train')
create_txt(annot_val,image_attri_val,'val')

In [ ]:
import os
print(len(os.listdir('dataset/train/labels')),len(os.listdir('dataset/train/images')))
print(len(os.listdir('dataset/val/labels')),len(os.listdir('dataset/val/images')))

In [ ]:
!pip install ultralytics

In [ ]:
names=[i['name'].split(',')[0].replace(' ','') for i in datatrain['categories'] if i['id'] in range(0,12)]
with open('data.yaml','w') as datafile:
  lines=['path: ../dataset/\n',
         'train: train/images\n',
         'val: val/images\n',
         'nc: '+str(len(names))+'\n',
         'names: '+str(names)]
  datafile.writelines(lines)
  datafile.close()

In [ ]:
del(datatrain)
del(dataval)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-seg.pt')
results = model.train(data='data.yaml',epochs=100, imgsz=640,  save_period=1, pretrained=True)